https://towardsdatascience.com/training-convolutional-neural-networks-to-categorize-clothing-with-pytorch-30b6d399f05f

In [0]:
#Importing libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [0]:
#Initializing hyperparameters
num_epochs = 8
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [0]:
#Loading dataset
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

train_dataset = datasets.FashionMNIST(root='./data', 
                            train=True, 
                            download=True,
                            transform=transform)

test_dataset = datasets.FashionMNIST(root='./data', 
                           train=False, 
                           download=True,
                           transform=transform)

In [0]:
#Loading dataset into dataloader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [0]:

#Defining the network          
class CNNModel(nn.Module):
    def __init__(self):
      super(CNNModel, self).__init__()

      #Convolution 1
      self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
      self.relu1 = nn.ReLU()

      #Max pool 1
      self.maxpool1 = nn.MaxPool2d(kernel_size=2)

      #Convolution 2
      self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
      self.relu2 = nn.ReLU()

      #Max pool 2
      self.maxpool2 = nn.MaxPool2d(kernel_size=2)

      #Dropout for regularization
      self.dropout = nn.Dropout(p=0.5)

      #Fully Connected 1
      self.fc1 = nn.Linear(32*7*7, 10)
 

    def forward(self, x):
        #Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        #Max pool 1
        out = self.maxpool1(out)
        
        #Convolution 2
        out = self.cnn2(out)
        out = self.relu2(out)
        
        #Max pool 2
        out = self.maxpool2(out)
        
        #Resize
        out = out.view(out.size(0), -1)
        
        #Dropout
        out = self.dropout(out)
        
        #Fully connected 1
        out = self.fc1(out)
        return out

In [0]:

#Create instance of model
model = CNNModel()

#Create instance of loss
criterion = nn.CrossEntropyLoss()


#Create instance of optimizer (Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [0]:
#Train the model
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        
        #Clear the gradients
        optimizer.zero_grad()
        
        #Forward propagation 
        outputs = model(images)      
        
        #Calculating loss with softmax to obtain cross entropy loss
        loss = criterion(outputs, labels)
        
        #Backward propation
        loss.backward()
        
        #Updating gradients
        optimizer.step()
        
        iter += 1
        
        #Total number of labels
        total = labels.size(0)
        
        #Obtaining predictions from max value
        _, predicted = torch.max(outputs.data, 1)
        
        #Calculate the number of correct answers
        correct = (predicted == labels).sum().item()
        
        #Print loss and accuracy
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, len(train_loader), loss.item(),
                          (correct / total) * 100))

Epoch [1/8], Step [100/600], Loss: 0.2836, Accuracy: 93.00%
Epoch [1/8], Step [200/600], Loss: 0.0931, Accuracy: 97.00%
Epoch [1/8], Step [300/600], Loss: 0.3168, Accuracy: 93.00%
Epoch [1/8], Step [400/600], Loss: 0.1845, Accuracy: 94.00%
Epoch [1/8], Step [500/600], Loss: 0.0837, Accuracy: 97.00%
Epoch [1/8], Step [600/600], Loss: 0.0639, Accuracy: 98.00%
Epoch [2/8], Step [100/600], Loss: 0.1042, Accuracy: 95.00%
Epoch [2/8], Step [200/600], Loss: 0.0489, Accuracy: 97.00%
Epoch [2/8], Step [300/600], Loss: 0.0373, Accuracy: 99.00%
Epoch [2/8], Step [400/600], Loss: 0.0488, Accuracy: 99.00%


In [0]:
#Testing the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = Variable(images)
        labels = Variable(labels)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
